# Модуль 1.1 — LLM vs ChatModel

**Цель:** понять разницу между моделью completion (`OpenAI`) и chat-моделью (`ChatOpenAI`) на простом примере.
Если completion‑модель у провайдера недоступна, используем локальную completion‑модель в Colab как альтернативу.

**Что сделаем:**
- установим нужные библиотеки прямо в блокноте
- настроим API‑ключ
- сравним ответы completion и chat
- (опционально) проверим локальную completion‑модель

In [ ]:
%pip -q install -U \
  langchain \
  langchain-openai \
  langchain-community \
  pydantic==2.12.3 \
  python-dotenv \
  tiktoken \
  requests==2.32.4

## Настройка ключа и base URL

В Colab можно задать `OPENAI_API_KEY` через ввод или использовать `.env` файл, если вы его загрузили в сессию.

Если вы ходите через AITunnel, задайте `OPENAI_BASE_URL` (например, `https://api.aitunnel.ru/v1/`) и используйте модели `gpt-5-mini` или `gpt-5-nano`.

In [ ]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Введите OPENAI_API_KEY: ")

# Для AITunnel (или другого OpenAI-совместимого прокси)
if not os.environ.get("OPENAI_BASE_URL"):
    os.environ["OPENAI_BASE_URL"] = "https://api.aitunnel.ru/v1/"

## Сравнение: completion vs chat

**Идея:** одна и та же задача, два разных интерфейса.

- `OpenAI` принимает строковый промпт и возвращает текст.
- `ChatOpenAI` работает с сообщениями (`HumanMessage`, `SystemMessage`) и дает больше контроля над ролями.

In [ ]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.messages import HumanMessage

prompt = "Сгенерируй 5 вариантов заголовка статьи про практическое применение LangChain."

# Можно переключать на более дешевые модели при необходимости скорости/стоимости
chat_model_name = "gpt-5.2-chat"  # альтернатива: "gpt-5-mini"

# Completion-модель зависит от провайдера (часто не поддерживается в прокси)
# Если у вас есть доступ к completion‑модели, укажите ее здесь
completion_model_name = None  # например: "gpt-3.5-turbo-instruct"

base_url = os.environ.get("OPENAI_BASE_URL")

# Chat-модель
chat = ChatOpenAI(
    model=chat_model_name,
    temperature=0.7,
    max_tokens=256,
    base_url=base_url,
)

chat_result = chat.invoke([HumanMessage(content=prompt)])

print("ChatModel:")
print(chat_result.content)

if completion_model_name:
    llm = OpenAI(
        model=completion_model_name,
        temperature=0.7,
        max_tokens=256,
        base_url=base_url,
    )
    llm_result = llm.invoke(prompt)
    print("\nLLM (completion):")
    print(llm_result)
else:
    print("\nLLM (completion): пропущено — у провайдера нет completion‑модели")

## Прайс‑гайд и переключение моделей

**Рекомендация по умолчанию:** `gpt-5.2-chat` — лучший баланс качества для обучения и объяснений.

**Когда переключаться на `gpt-5-mini`:**
- много быстрых итераций (черновики, мозговые штурмы)
- простой контент без сложных рассуждений
- нужно снизить цену при сохранении приличного качества

**Когда переключаться на `gpt-5-nano`:**
- массовые прогоны, дешевые проверки
- короткие ответы и шаблонные задачи
- важна скорость и минимальная стоимость

**Пример цен (по списку провайдера):**
- `gpt-5.2-chat`: 33.6 ₽ / 1M входных токенов
- `gpt-5-mini`: 4.8 ₽ / 1M входных токенов
- `gpt-5-nano`: 0.96 ₽ / 1M входных токенов

**Как переключить модель:**

```python
chat_model_name = "gpt-5.2-chat"  # или "gpt-5-mini" / "gpt-5-nano"
```

## Опционально: локальная completion‑модель в Colab

Если ваш провайдер не поддерживает completion‑модели, можно использовать локальную causal‑LM как аналог completion.
Ниже — минимальный пример через `transformers`.

> Примечание: модель будет скачана в среду Colab, это может занять несколько минут.

In [ ]:
%pip -q install -U transformers accelerate sentencepiece

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

local_model_name = "ai-forever/rugpt3small_based_on_gpt2"

local_tokenizer = AutoTokenizer.from_pretrained(local_model_name)
local_model = AutoModelForCausalLM.from_pretrained(
    local_model_name,
    torch_dtype="auto",
    device_map="auto",
)

local_generator = pipeline(
    "text-generation",
    model=local_model,
    tokenizer=local_tokenizer,
)

local_prompt = "Сгенерируй 3 варианта заголовка статьи про практическое применение LangChain."
local_result = local_generator(local_prompt, max_new_tokens=80, do_sample=True, temperature=0.7)

print(local_result[0]["generated_text"])